# Чтение данных

Имеется набор данных: цветные фотографии, сделанные внутри помещений и на улице. Необходимо обучить модель, которая будет проводить классификацию изображений по этому признаку. Базовая модель будет построена следующим образом: по каждому изображению по каждому цвету будет построена гистограмма интенсивности этого цвета в пикселях изображения. На гистограмме будут отмечены нижний и верхний квартили и медиана. Таким образом, для каждого изображения будет получено девять признаков, по которым мы попробуем построить различные модели: логистическую регрессию, метод опорных, $K$ ближайших соседей (или, быть может, метод парзеновского окна). В этом файле мы подготовим два набора данных: 

1) изображения, отмасштабированные к виду $50 \times 50$ пикселей
2) квартили, построенные по полученным изображениям

Импортируем инструменты

In [1]:
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from skimage.io import imread_collection
from skimage.transform import resize

from tqdm import tqdm

Загружаем данные и меняем размер изображений

In [2]:
%%time

indoor = imread_collection("images\indoor\*.jpg")

indoor_ndarray = np.array(
    [resize(indoor.load_func(filename), (50, 50)) for filename in tqdm(indoor.files)], 
    dtype=np.float16
)

indoor_ndarray.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28106/28106 [07:30<00:00, 62.35it/s]


Wall time: 7min 40s


(28106, 50, 50, 3)

In [3]:
%%time

outdoor = imread_collection("images\outdoor\*.jpg")

outdoor_ndarray = np.array(
    [resize(outdoor.load_func(filename), (50, 50)) for filename in tqdm(outdoor.files)], 
    dtype=np.float16
)

outdoor_ndarray.shape

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27216/27216 [07:34<00:00, 59.85it/s]


Wall time: 7min 43s


(27216, 50, 50, 3)

Из полученных массивов формируем таблицу размером `len(indoor) + len(outdoor)`$\times 7500$, где $7500 = 50 \times 50 \times 3$. То есть, для каждого изображения мы составляем строку, в которой сначала идут красные компоненты каждого пикселя, затем зелёные и наконец синиме. 

In [4]:
%%time

indoor_rows = np.hstack((
    indoor_ndarray[:, :, :, 0].reshape(len(indoor_ndarray), -1), 
    indoor_ndarray[:, :, :, 1].reshape(len(indoor_ndarray), -1), 
    indoor_ndarray[:, :, :, 2].reshape(len(indoor_ndarray), -1),
    np.ones(shape=(len(indoor_ndarray), 1), dtype=np.float16)
))

outdoor_rows = np.hstack((
    outdoor_ndarray[:, :, :, 0].reshape(len(outdoor_ndarray), -1), 
    outdoor_ndarray[:, :, :, 1].reshape(len(outdoor_ndarray), -1), 
    outdoor_ndarray[:, :, :, 2].reshape(len(outdoor_ndarray), -1),
    np.zeros(shape=(len(outdoor_ndarray), 1), dtype=np.float16)
))

color_names = ["red", "green", "blue"]
columns=[f"{color}_{i}_{j}" for color in color_names for i in range(50) for j in range(50)]
columns.append("target")

images = pd.DataFrame(np.vstack((indoor_rows, outdoor_rows)), columns=columns)
images.info()
images.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55322 entries, 0 to 55321
Columns: 7501 entries, red_0_0 to target
dtypes: float16(7501)
memory usage: 791.5 MB
Wall time: 1.35 s


,red_0_0,red_0_1,red_0_2,red_0_3,red_0_4,red_0_5,red_0_6,red_0_7,red_0_8,red_0_9,...,blue_49_41,blue_49_42,blue_49_43,blue_49_44,blue_49_45,blue_49_46,blue_49_47,blue_49_48,blue_49_49,target
0,0.117676,0.117676,0.117676,0.117676,0.117676,0.117676,0.117676,0.117676,0.117676,0.117676,...,0.572754,0.554199,0.442627,0.434814,0.451172,0.470459,0.487793,0.505859,0.534180,1.0
1,0.846191,0.865234,0.971680,0.981934,0.987305,0.972656,0.990723,0.954102,0.883789,0.947266,...,0.035309,0.003922,0.027451,0.075989,0.111267,0.114197,0.103943,0.096069,0.075500,1.0
2,0.974121,0.951172,0.700195,0.699707,0.750977,0.737793,0.982910,0.921875,0.877441,0.683105,...,0.894043,0.851074,0.866699,0.886719,0.890137,0.890137,0.890137,0.890137,0.890137,1.0
3,0.723633,0.723633,0.650879,0.719727,0.747070,0.770508,0.780273,0.772461,0.735352,0.723633,...,0.519043,0.513184,0.508789,0.535156,0.304443,0.023300,0.033020,0.081177,0.232300,1.0
4,0.799316,0.810547,0.820312,0.804688,0.770996,0.742188,0.724609,0.712402,0.703613,0.702148,...,0.297607,0.471436,0.501465,0.489990,0.433350,0.377930,0.437744,0.484863,0.474854,1.0


Теперь считаем квартили

In [5]:
%%time

quartiles = np.linspace(0.25, 1, 3, endpoint=False)

indoor_quartiles = np.quantile(indoor_ndarray, quartiles, axis=(1, 2))
indoor_quartiles = np.hstack((
    indoor_quartiles[0], 
    indoor_quartiles[1], 
    indoor_quartiles[2], 
    np.ones(shape=(len(indoor_quartiles[0]), 1))
))

outdoor_quartiles = np.quantile(outdoor_ndarray, quartiles, axis=(1, 2))
outdoor_quartiles = np.hstack((
    outdoor_quartiles[0], 
    outdoor_quartiles[1], 
    outdoor_quartiles[2], 
    np.zeros(shape=(len(outdoor_quartiles[0]), 1))
))

quartile_names = ["upper_quartiles", "median", "lower_quartiles"]
columns = [f"{color}_{quartile}" for color in color_names for quartile in quartile_names]
columns.append("target")

images_quartiles = pd.DataFrame(np.vstack((indoor_quartiles, outdoor_quartiles)), columns=columns)
images_quartiles.info()
images_quartiles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55322 entries, 0 to 55321
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   red_upper_quartiles    55322 non-null  float64
 1   red_median             55322 non-null  float64
 2   red_lower_quartiles    55322 non-null  float64
 3   green_upper_quartiles  55322 non-null  float64
 4   green_median           55322 non-null  float64
 5   green_lower_quartiles  55322 non-null  float64
 6   blue_upper_quartiles   55322 non-null  float64
 7   blue_median            55322 non-null  float64
 8   blue_lower_quartiles   55322 non-null  float64
 9   target                 55322 non-null  float64
dtypes: float64(10)
memory usage: 4.2 MB
Wall time: 15.1 s


,red_upper_quartiles,red_median,red_lower_quartiles,green_upper_quartiles,green_median,green_lower_quartiles,blue_upper_quartiles,blue_median,blue_lower_quartiles,target
0,0.195892,0.166290,0.152954,0.379395,0.333496,0.317017,0.617920,0.583008,0.556274,1.0
1,0.741699,0.657227,0.434692,0.803223,0.729492,0.527344,0.847290,0.774902,0.595215,1.0
2,0.686035,0.567139,0.567749,0.832275,0.811523,0.804443,0.890137,0.874512,0.867798,1.0
3,0.466248,0.420166,0.320251,0.561768,0.504395,0.410400,0.640625,0.565063,0.477539,1.0
4,0.654297,0.659912,0.615234,0.757324,0.750000,0.696289,0.810547,0.802734,0.752930,1.0


Сохраняем результаты

In [6]:
pd.DataFrame(images.values, columns=images.columns, dtype=np.float64).to_parquet("images.parquet", compression=None)

1

In [7]:
images_quartiles.to_parquet("images_quartiles.parquet", compression=None)

1